# xDrip+ Data Analysis for Gastroparesis Screening (XGastro-EDA)

## Introduction
This notebook demonstrates the processing and analysis of Continuous Glucose Monitoring (CGM) data from xDrip+ for potential gastroparesis screening applications. By carefully processing CGM data alongside meal and insulin records, we create a foundation for analyzing post-meal glucose response patterns.

### Project Context
Gastroparesis, a condition affecting gastric emptying, can be challenging to diagnose. Traditional screening methods often require specialized testing. This project explores the potential of using readily available CGM data to identify patterns that might indicate delayed gastric emptying.

### Data Source
- **xDrip+**: An open-source diabetes management application
- **Time Period**: June 2023 - September 2024
- **Key Components**:
 * CGM readings at 5-minute intervals
 * Meal records with carbohydrate quantities
 * Insulin dosing (basal and bolus)

### Project Objectives
1. **Data Integration**
  - Align CGM, meal, and insulin data to common timeline
  - Ensure data quality and completeness
  - Create robust meal analysis framework

2. **Quality Assurance**
  - Identify and characterize data gaps
  - Implement appropriate interpolation strategies
  - Maintain data integrity for clinical analysis

3. **Analysis Preparation**
  - Focus on significant meals (>20g carbs)
  - Create 4-hour post-meal analysis windows
  - Develop quality metrics for meal data assessment

### Notebook Structure
1. Data extraction and cleaning
2. Timeline alignment and gap analysis
3. Meal period identification and classification
4. Quality assessment and interpolation
5. Statistical validation
6. Preparation for further analysis

This notebook focuses on data preparation and quality assessment, laying the groundwork for subsequent analysis of post-meal glucose patterns.

# Required Libraries and Dependencies

This notebook uses several Python libraries for data manipulation, analysis, and visualization:

- **pandas**: Core data manipulation and analysis
- **numpy**: Numerical operations and array handling (imported in respective module)
- **matplotlib**: Data visualization and plotting
- **sqlalchemy**: Database connectivity and operations
- **ast**: Safe parsing of JSON-like strings (imported in respective module)
- **enum, dataclass, typing**: Type hinting and structured data classes (imported in respective module)
- **display**: Improved print options in Jupyter
- **pprint**: Improved print layout of nested data structures

Note: This project assumes you have these libraries installed. If needed, install via pip:
```pip install pandas numpy matplotlib sqlalchemy```

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from IPython.display import display
from pprint import pprint

# Import our created functions

You can check all these out in the src directory.

In [4]:
import os
import sys
notebook_path = os.path.abspath('.')
project_root = os.path.join(notebook_path, '../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.preprocessing.cleaning import clean_classify_insulin, clean_classify_carbs, clean_glucose
from src.preprocessing.alignment import align_diabetes_data
from src.analysis.config import MealQuality, MealAnalysisConfig
from src.analysis.gaps import analyse_glucose_gaps
from src.analysis.insulin import analyse_insulin_over_time
from src.analysis.meals import analyse_meal_data
from src.visualization.formatting import format_gaps_output, format_meal_statistics

ModuleNotFoundError: No module named 'src.preprocessing.alignment'

# Data Source: xDrip+ SQLite Database

This notebook begins by connecting to an xDrip+ SQLite database export. xDrip+ is an open-source diabetes management application that stores various diabetes-related data including glucose readings, treatments, and device status information.

## Database Connection
We establish a connection to the SQLite database and inspect its structure. The database contains 29 tables storing different aspects of diabetes management data:

Key tables for our analysis:
- **BgReadings**: Continuous glucose monitoring (CGM) data
- **Treatments**: Insulin doses and carbohydrate intake records
- **Calibration**: Sensor calibration events
- **Sensors**: CGM sensor information

```python

In [ ]:
# Path to your SQLite file
db_path = 'data/export20240928-130349.sqlite'

# Create an SQLAlchemy engine
engine = create_engine(f'sqlite:///{db_path}')

# Use SQLAlchemy's inspector to list all tables
inspector = inspect(engine)
tables = inspector.get_table_names()

# Display firmatted table names
#displayer.display_database_tables(db_path, tables)
pprint(tables)

# Loading and Initial Data Processing

We extract three key datasets from the xDrip+ database and prepare them for analysis by:
1. Converting Unix timestamps to pandas datetime format
2. Setting timestamps as index for time-series analysis
3. Creating separate dataframes for:
  - Blood glucose readings
  - Treatment records (insulin & carbohydrates)
  - Heart rate measurements

## Data Loading Process
Each table requires the same preprocessing steps:
- SQL to pandas DataFrame conversion
- Timestamp conversion from milliseconds
- Index creation from timestamp

Let's examine the structure of our blood glucose data:

In [ ]:
# Load BgReadings table into a pandas DataFrame
glucose_data = 'BgReadings'  # Table containing all BG Readings from XDrip+
bg_df = pd.read_sql_table(glucose_data, con=engine)
bg_df['timestamp'] = pd.to_datetime(bg_df['timestamp'], unit='ms')
bg_df.set_index('timestamp', inplace=True)

# Load Treatments table into a pandas DataFrame
treatment_data = 'Treatments'  # Table containing all Treatments from XDrip+
treatment_df = pd.read_sql_table(treatment_data, con=engine)
treatment_df['timestamp'] = pd.to_datetime(treatment_df['timestamp'], unit='ms')
treatment_df.set_index('timestamp', inplace=True)

# Load HeartRate table into a pandas Dataframe (Not required for this analysis)
heart_rate_data = 'HeartRate' # Table containing HeartRate data
heart_rate_df = pd.read_sql_table(heart_rate_data, con=engine)
heart_rate_df['timestamp'] = pd.to_datetime(heart_rate_df['timestamp'], unit='ms')
heart_rate_df.set_index('timestamp', inplace=True)

# Explore the first few rows of the blood glucose table
bg_df.head()

# Treatment Data Preprocessing: Insulin Classification

This function implements a sophisticated insulin classification algorithm for diabetes treatment data. The classification process handles both labeled and unlabeled insulin entries, using a combination of explicit insulin types and quantity-based rules.

## Classification Rules
1. **Labeled Insulin**
  - Novorapid → Bolus insulin
  - Levemir → Basal insulin

2. **Unlabeled Insulin**
  - ≤ 8 units → Classified as Bolus
  - 8-15 units → Classified as Basal
  - >15 units → Dropped (likely data entry errors)

## Function Process
- Creates separate columns for bolus and basal insulin
- Processes JSON metadata for labeled entries
- Applies rule-based classification for unlabeled entries
- Maintains data integrity by dropping suspicious entries

In [ ]:
def clean_classify_insulin(df):
    '''
    Clean and separate basal and bolus insulin into separate columns based on insulin type if selected.
    "Novorapid" = Bolus & "Levemir" = Basal, if undefined then all insulin <= 8 units are classified as bolus,
    and <= 15 units are classified as Basal. All unclassified treatments over 15 units are dropped.
    '''
    
    # Create a copy to avoid altering the original dataframe
    df_clean = df.copy()
    
    # Keep only rows where insulin is > 0.0 units
    df_clean = df_clean[df_clean['insulin'] > 0.0]
    
    # Initialize bolus and basal columns with 0
    df_clean['bolus'] = 0.0
    df_clean['basal'] = 0.0
    
    # Process labeled data first using insulinJSON column
    def extract_insulin_type(row):
        try:
            if pd.isna(row['insulinJSON']):
                return None
            data = json.loads(row['insulinJSON'])
            insulin_type = data.get('insulin', '').lower()
            if 'novorapid' in insulin_type:
                df_clean.at[row.name, 'bolus'] = row['insulin']
            elif 'levemir' in insulin_type:
                df_clean.at[row.name, 'basal'] = row['insulin']
        except:
            return None
        
    # Apply extract_insulin_types to each row
    df_clean.apply(extract_insulin_type, axis=1)
    
    # Process unlabeled data
    unlabeled_mask = (df_clean['bolus'] == 0) & (df_clean['basal'] == 0)
    
    # Drop all rows where insulin is unclassified AND > 15 units (Likely error)
    df_clean = df_clean.drop(
        df_clean[unlabeled_mask & (df_clean['insulin'] > 15)].index
    )
    
    # Classify remaining unlabeled data
    df_clean.loc[unlabeled_mask & (df_clean['insulin'] <= 8), 'bolus'] = df_clean['insulin']
    df_clean.loc[unlabeled_mask & (df_clean['insulin'] > 8) & (df_clean['insulin'] <= 15), 'basal'] = df_clean['insulin']
    
    # Drop the original insulin column if desired
    # df_clean = df_clean.drop('insulin', axis=1)
    
    return df_clean

In [ ]:
insulin_df = clean_classify_insulin(treatment_df)

# Analysing Insulin Usage Patterns Over Time

This analysis examines the temporal patterns in insulin usage, providing insights into treatment consistency and data quality. The function performs both data analysis and visualization, helping identify trends and potential data collection issues.

## Analysis Components
1. **Temporal Aggregation**
  - Groups data by month
  - Processes JSON metadata for insulin types
  - Calculates usage percentages for different insulin types

2. **Insulin Classification**
  - Novorapid (rapid-acting insulin)
  - Levemir (long-acting insulin)
  - Other insulin types
  - Unspecified entries

## Visualisation
The function generates two complementary plots:
1. **Stacked Area Chart**
  - Shows relative distribution of insulin types
  - Uses consistent color coding (Orange: Novorapid, Green: Levemir)
  - Reveals changes in recording practices

2. **Entry Count Timeline**
  - Displays total monthly entries
  - Helps identify data collection gaps or inconsistencies

In [ ]:
def analyse_insulin_over_time(df):
    # Create a copy of the dataframe with monthly periods
    df_monthly = df.copy()
    df_monthly['month'] = df_monthly.index.to_period('M')
    
    # Function to process insulin types from JSON
    def get_insulin_types(entry):
        if pd.isna(entry) or entry == "[]":
            return ["Not Stated"]
        insulin_data = ast.literal_eval(entry)
        return [record['insulin'] for record in insulin_data]
    
    # Create monthly statistics
    monthly_stats = []
    
    # Get unique months and sort them
    unique_months = sorted(df_monthly['month'].unique())
    
    for month in unique_months:
        month_data = df_monthly[df_monthly['month'] == month]
        
        # Count insulin types for this month
        insulin_counts = {'Novorapid': 0, 'Levemir': 0, 'Other': 0, 'Not Stated': 0}
        total_entries = len(month_data)
        
        for entry in month_data['insulinJSON']:
            insulin_types = get_insulin_types(entry)
            for insulin_type in insulin_types:
                if insulin_type == "Novorapid":
                    insulin_counts['Novorapid'] += 1
                elif insulin_type == "Levemir":
                    insulin_counts['Levemir'] += 1
                elif insulin_type == "Not Stated":
                    insulin_counts['Not Stated'] += 1
                else:
                    insulin_counts['Other'] += 1
        
        # Calculate percentages
        monthly_stats.append({
            'month': str(month),  # Convert period to string
            'Novorapid': (insulin_counts['Novorapid'] / total_entries) * 100,
            'Levemir': (insulin_counts['Levemir'] / total_entries) * 100,
            'Other': (insulin_counts['Other'] / total_entries) * 100,
            'Not Stated': (insulin_counts['Not Stated'] / total_entries) * 100,
            'total_entries': total_entries
        })
    
    # Convert to DataFrame
    stats_df = pd.DataFrame(monthly_stats)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(25, 15))
    
    # Stacked area chart
    ax1.stackplot(range(len(stats_df)), 
                 [stats_df['Novorapid'].values, 
                  stats_df['Levemir'].values,
                  stats_df['Other'].values, 
                  stats_df['Not Stated'].values],
                 labels=['Novorapid', 'Levemir', 'Other', 'Not Stated'],
                 colors=['#F26F21', '#35b57d', '#808080', '#D3D3D3'])  # Add specific colors

                 
    
    # Set x-axis ticks and labels
    ax1.set_xticks(range(len(stats_df)))
    ax1.set_xticklabels(stats_df['month'], rotation=45)
    ax1.set_title('Distribution of Insulin Types Over Time')
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Percentage')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Line plot of total entries
    ax2.plot(range(len(stats_df)), stats_df['total_entries'].values, marker='o')
    ax2.set_xticks(range(len(stats_df)))
    ax2.set_xticklabels(stats_df['month'], rotation=45)
    ax2.set_title('Total Number of Entries per Month')
    ax2.set_xlabel('Month')
    ax2.set_ylabel('Number of Entries')
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    return plt, stats_df

# Usage:
plot, insulin_counts = analyse_insulin_over_time(insulin_df)
plot.show()

# Data Validation: Raw Insulin Distribution

Before proceeding with our cleaned and classified insulin data, we visualize the raw insulin entries to:
1. Verify the range of insulin values
2. Identify potential outliers
3. Check temporal distribution of entries
4. Validate our classification thresholds (8u for bolus, 15u for basal)

The scatter plot provides a quick visual validation that our insulin classification function's assumptions align with the actual data patterns.

In [ ]:
# Make plot 25 x 12
plt.figure(figsize=(25, 12))

plt.plot(insulin_df['insulin'], label='Insulin', marker='o', linestyle = 'None', color='#F26F21')

# Add labels, title, and legend
plt.xlabel('Timestamp')
plt.ylabel('Insulin')
plt.legend()

plt.show()

In [ ]:
# Make plot 25 x 12
plt.figure(figsize=(25, 12))

# Create masks for NovoRapid (bolus) and Levemir (basal)
bolus_mask = insulin_df['bolus'] >0.0
basal_mask = insulin_df['basal']> 0.0

# Plot bolus (NovoRapid) points
plt.plot(
    np.array(insulin_df[bolus_mask].index),  # Convert index to NumPy array
    np.array(insulin_df[bolus_mask]['insulin']),  # Convert insulin values to NumPy array
    label='NovoRapid (Bolus)', 
    marker='o', 
    linestyle='None',  # No line connecting points
    color='#F26F21'  # NovoRapid orange
)

# Plot basal (Levemir) points
plt.plot(
    np.array(insulin_df[basal_mask].index),  # Convert index to NumPy array
    np.array(insulin_df[basal_mask]['insulin']),  # Convert insulin values to NumPy array
    label='Levemir (Basal)', 
    marker='o', 
    linestyle='None',  # No line connecting points
    color='#35b57d'  # Levemir green
)

# Add labels, title, and legend
plt.xlabel('Timestamp')
plt.ylabel('Insulin')
plt.legend()

plt.show()

# Refined Insulin Dataset Structure

After cleaning and classification, we isolate our two key insulin variables (bolus and basal) and examine the resulting dataset structure. This inspection helps:

1. **Verify Data Processing**
  - Confirm successful separation of insulin types
  - Check data types and memory usage
  - Review null values if any

2. **Statistical Overview**
  - Distribution of both insulin types
  - Typical dosing ranges
  - Identify any remaining outliers

The .info() shows our dataframe structure while .describe() provides key statistical measures.


In [ ]:
insulin_df = insulin_df[['bolus', 'basal']]

In [ ]:
insulin_df.info()

In [ ]:
insulin_df.describe()

# Carbohydrate Data Preprocessing

A straightforward but essential cleaning step for carbohydrate data. This function:
1. Creates a safe copy of the original data
2. Removes trivial carbohydrate entries (<1g)
3. Maintains original timestamp indexing

## Rationale
- Entries below 1g are likely recording errors or insignificant for meal analysis
- Clean dataset focuses on meaningful carbohydrate intake events
- Preserves original data structure for later meal impact analysis

In [ ]:
def clean_classify_carbs(df):
    
    # Create a copy to avoid altering the original dataframe
    df_clean = df.copy()
    
    # Keep only rows where carbs is >= 1.0 grams
    df_clean = df_clean[df_clean['carbs'] >= 1.0]
    
    
    
    
    return df_clean

carb_df = clean_classify_carbs(treatment_df)

# Refined Carbohydrate Dataset Structure

After cleaning our carbohydrates dataset, we simply wish to keep the index and carbs column.


In [ ]:
# Drop all columns except carbohydrate
carb_df = carb_df[['carbs']]
carb_df

# Visual Validation: Carbohydrate Distribution

Following our carbohydrate data cleaning, we visualize the temporal distribution and magnitude of carbohydrate entries to:

1. Confirm removal of sub-1g entries
2. Examine the range of carbohydrate values
3. Check recording frequency and patterns
4. Identify any potential outliers requiring attention

The scatter plot representation helps validate our minimal cleaning approach and provides insight into meal recording patterns.

In [ ]:
# Make plot 25 x 12
plt.figure(figsize=(25, 12))

plt.plot(carb_df['carbs'], label='Carbohydrates', marker='o', linestyle = 'None')

# Add labels, title, and legend
plt.xlabel('Timestamp')
plt.ylabel('Carbohydrates')
plt.legend()

plt.show()

# Carbohydrate Dataset Summary

After basic cleaning, we examine the carbohydrate dataset structure and statistics to:

1. **Verify Dataset Properties**
  - Confirm single carbohydrate column retention
  - Check data type consistency
  - Review number of recorded meals

2. **Statistical Distribution**
  - Typical meal carbohydrate content
  - Range of carbohydrate values
  - Help validate typical meal size assumptions

The .info() provides structure verification while .describe() shows the statistical distribution of meal carbohydrates.


In [ ]:
carb_df.info()

In [ ]:
carb_df.describe()

# Blood Glucose Data Preprocessing

This cleaning function prepares blood glucose data for analysis by implementing essential clinical and analytical standards:

## Cleaning Steps
1. **Unit Standardization**
  - Renames 'calculated_value' to 'mg_dl' for clarity
  - Creates 'mmol_l' column (standard international units)
  - Conversion factor: mg/dL × 0.0555 = mmol/L

2. **Clinical Range Enforcement**
  - Lower bound: 39.64 mg/dL (2.2 mmol/L)
  - Upper bound: 360.36 mg/dL (20.0 mmol/L)
  - Values outside range clipped to these limits

3. **Data Simplification**
  - Retains only essential columns
  - Preserves timestamp index
  - Maintains dual unit representation

## Rationale
- Range limits reflect typical CGM sensor capabilities
- Dual units support international analysis
- Simplified structure focuses on key metrics

In [ ]:
def clean_glucose(df):
    
    # Create copy to avoid altering original dataframe
    clean_df = df.copy()
    
    # Rename the 'calculated_value' column to 'mg_dl'
    clean_df.rename(columns={'calculated_value': 'mg_dl'}, inplace=True)
    
    # Limit the 'mg_dl' column values to the range 39.64 to 360.36
    clean_df['mg_dl'] = clean_df['mg_dl'].clip(lower=39.64, upper=360.36)

    
    # Create a new column 'mmol_l' by converting 'calculated_value' from mg/dL to mmol/L
    clean_df['mmol_l'] = clean_df['mg_dl'] * 0.0555
    
    # Drop all rows except mg_dl and mmol_l
    clean_df = clean_df[['mg_dl', 'mmol_l' ]]
    
    return clean_df

glucose_df = clean_glucose(bg_df)

In [ ]:
glucose_df

# Blood Glucose Data Visualization

We examine the glucose data through two complementary visualizations to validate our cleaning process and understand data patterns:

## Full Dataset Scatter Plot
- Shows complete temporal distribution
- Validates clipping of extreme values
- Reveals overall data density
- Highlights any major gaps

## Recent 90-Day Time Series
- Focused view of latest 90 days (25,920 points at 5-min intervals)
- Connected line plot shows glucose trends
- More detailed examination of recent monitoring
- Better visualization of daily patterns

The contrast between these views helps validate both long-term data collection and recent monitoring quality.

In [ ]:
# Make plot 25 x 12
plt.figure(figsize=(25, 12))

plt.plot(glucose_df['mmol_l'], label='mmol/L', marker='o', linestyle = 'None')

# Add labels, title, and legend
plt.xlabel('Timestamp')
plt.ylabel('Blood Glucose (mmol/L)')
plt.legend()

plt.show()

In [ ]:
# Display most recent 90 days of glucose readings in mmol/L
last_90 = glucose_df[-25920:]

# Make plot 25 x 12
plt.figure(figsize=(25, 12))

plt.plot(last_90['mmol_l'], label='mmol/L')

# Add labels, title, and legend
plt.title('Glucose readings in mmol/L - Most recent 90 Days')
plt.xlabel('Timestamp')
plt.ylabel('Blood Glucose (mmol/L)')
plt.legend()

plt.show()

In [ ]:
# Display most recent 48 Hours of glucose readings in mmol/L
last_48h = glucose_df[-576:]

# Make plot 25 x 12
plt.figure(figsize=(25, 12))

plt.plot(last_48h['mmol_l'], label='mmol/L')

# Add labels, title, and legend
plt.title('Glucose readings in mmol/L - Most recent 48 Hours')
plt.xlabel('Timestamp')
plt.ylabel('Blood Glucose (mmol/L)')
plt.legend()

plt.show()

We can see that the last 48 hour graph shows messy lines especially around 3 am, this could be due to incosistent gap sizes causing the lines to dilate and contract depending on the size of the gap. We will visualise this again after aligning our datasets to regular 5-minute intervals.

In [ ]:
glucose_df.info()

In [ ]:
glucose_df.describe()

# Dataset Overview: Final Check

Before proceeding with dataset alignment, we perform a final verification of our three core datasets. This check ensures:

1. **Data Dimensions**
  - Number of records in each dataset
  - Column confirmations
  - Memory usage

2. **Time Range Verification**
  - All datasets properly indexed by timestamp
  - Presence of complete data columns
  - No unexpected alterations from cleaning

3. **Data Completeness**
  - Blood glucose monitoring frequency
  - Meal (carbohydrate) records
  - Insulin treatment entries

In [ ]:
glucose_df.info()

In [ ]:
carb_df.info()

In [ ]:
insulin_df.info()

# Data Alignment and Synthesis

After cleaning individual datasets, we now combine them into a unified timeframe with consistent 5-minute intervals. This critical step creates our foundation for meal impact analysis.

## Alignment Process
1. **Timestamp Standardization**
  - Rounds all timestamps to nearest 5-minute mark
  - Creates uniform timeline across all data types
  - Spans from earliest to latest record

2. **Data Integration**
  - Blood Glucose: Averaged within intervals
  - Carbohydrates: Summed within intervals
  - Insulin: Bolus and basal summed separately

3. **Quality Assurance**
  - Shape verification
  - Column completeness
  - Statistical distribution check

The resulting dataset provides:
- Complete timeline with no gaps
- Aligned treatment and response data
- Missing value handling appropriate to data type
 * Glucose: NaN preserved
 * Treatments: 0 for missing values

In [ ]:
def align_diabetes_data(
    bg_df: pd.DataFrame,
    carb_df: pd.DataFrame,
    insulin_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Aligns diabetes data (blood glucose, carbohydrates, insulin) to regular 5-minute intervals.
    
    This function takes three dataframes with timestamped diabetes data and produces a single
    dataframe with regular 5-minute intervals. Blood glucose readings are averaged within
    each interval, while carbohydrates and insulin entries are summed.
    
    Args:
        bg_df: DataFrame with timestamp index and columns ['mg_dl', 'mmol_l']
              Blood glucose readings at irregular intervals
        carb_df: DataFrame with timestamp index and column ['carbs']
                Sporadic carbohydrate entries
        insulin_df: DataFrame with timestamp index and columns ['bolus', 'basal']
                   Sporadic insulin entries
    
    Processing steps:
    1. Round all timestamps to nearest 5-minute interval
    2. Create complete timeline at 5-minute intervals from earliest to latest data point
    3. Resample BG data - average any readings within each 5-min window
    4. Resample treatment data - sum any entries within each 5-min window
    5. Combine all data and ensure all intervals exist
    6. Fill missing treatment values with 0 (keep BG as NaN)
    
    Returns:
        DataFrame with:
        - Regular 5-minute interval index
        - Columns: ['mg_dl', 'mmol_l', 'carbs', 'bolus', 'basal']
        - BG values averaged within intervals, NaN where missing
        - Treatment values summed within intervals, 0 where missing
    """
    # First round timestamps in all dataframes to 5-min intervals
    bg_df = bg_df.copy()
    carb_df = carb_df.copy()
    insulin_df = insulin_df.copy()
    
    bg_df.index = bg_df.index.round('5min')
    carb_df.index = carb_df.index.round('5min')
    insulin_df.index = insulin_df.index.round('5min')
    
    
    # Create complete timeline of 5-min intervals
    full_range = pd.date_range(
        start=min(bg_df.index.min(), carb_df.index.min(), insulin_df.index.min()),
        end=max(bg_df.index.max(), carb_df.index.max(), insulin_df.index.max()),
        freq='5min'
    )
    
    
    # Resample blood glucose - average within windows
    bg_resampled = bg_df.resample('5min').agg({
        'mg_dl': 'mean',
        'mmol_l': 'mean'
    })
    
    # Resample carbs - sum within windows
    carb_resampled = carb_df.resample('5min').agg({
        'carbs': 'sum'
    })
    
    # Resample insulin - sum within windows
    insulin_resampled = insulin_df.resample('5min').agg({
        'bolus': 'sum',
        'basal': 'sum'
    })
    
    # Combine all data
    aligned_df = pd.concat([bg_resampled, carb_resampled, insulin_resampled], axis=1)
    
    # Ensure all 5-minute intervals exist
    aligned_df = aligned_df.reindex(full_range)
    
    # Fill missing treatment values with 0
    aligned_df['carbs'] = aligned_df['carbs'].fillna(0)
    aligned_df['bolus'] = aligned_df['bolus'].fillna(0)
    aligned_df['basal'] = aligned_df['basal'].fillna(0)

    
    return aligned_df

In [ ]:
combined_df = align_diabetes_data(glucose_df, carb_df, insulin_df)

In [ ]:
combined_df.shape

In [ ]:
combined_df.info()

In [ ]:
combined_df.describe()

# Glucose Data Gap Analysis

After alignment, we analyze the continuity of glucose monitoring data to identify and characterize monitoring gaps. This analysis is crucial for:

## Gap Detection Process
1. **Identification**
  - Finds sequences of missing glucose values
  - Records start and end times
  - Calculates gap durations

2. **Classification**
  - Stores gap information in structured format
  - Identifies longest gaps
  - Creates human-readable duration format

3. **Meal Impact Analysis**
  - Checks for meals preceding gaps
  - Flags gaps that might affect meal analysis
  - 4-hour pre-gap meal window check

## Key Outputs
- Top N largest gaps with detailed timing
- Total number of gaps
- Complete gap inventory
- Meal proximity warnings

In [ ]:
def analyse_glucose_gaps(aligned_df: pd.DataFrame, show_top_n: int = 10) -> dict:
    """
    Analyzes gaps in glucose readings with detailed information about largest gaps.
    
    Args:
        aligned_df: DataFrame with 5-min interval index and columns including
                   'mg_dl' and 'mmol_l'
        show_top_n: Number of largest gaps to show details for
    
    Returns:
        Dictionary containing gap analysis including detailed info about largest gaps
    """
    missing_mask = aligned_df['mg_dl'].isna()
    
    gap_starts = []
    gap_lengths = []
    gap_ends = []  # Added to track end times
    current_start = None
    
    for idx, (time, is_missing) in enumerate(missing_mask.items()):
        if is_missing and current_start is None:
            current_start = time
        elif not is_missing and current_start is not None:
            gap_starts.append(current_start)
            gap_ends.append(time)  # Store end time
            gap_lengths.append((time - current_start).total_seconds() / 60)
            current_start = None
    
    # Handle case where dataset ends with a gap
    if current_start is not None:
        gap_starts.append(current_start)
        gap_ends.append(missing_mask.index[-1])
        gap_lengths.append((missing_mask.index[-1] - current_start).total_seconds() / 60)
    
    # Create DataFrame with gap details
    gaps_df = pd.DataFrame({
        'start_time': gap_starts,
        'end_time': gap_ends,
        'length_minutes': gap_lengths
    })
    
    # Sort by gap length and get details of largest gaps
    largest_gaps = gaps_df.sort_values('length_minutes', ascending=False).head(show_top_n)
    
    # Add human-readable duration
    largest_gaps['duration'] = largest_gaps.apply(
        lambda x: f"{int(x['length_minutes'] // 60)}h {int(x['length_minutes'] % 60)}m", 
        axis=1
    )
    
    return {
        'total_gaps': len(gaps_df),
        'largest_gaps': largest_gaps,
        'gaps_df': gaps_df
    }

In [ ]:
def format_gaps_output(gaps_data):
    """
    Pretty-prints the gaps data in a Jupyter Notebook.
    
    Args:
        gaps_data (dict): Dictionary containing 'total_gaps', 'largest_gaps', and 'gaps_df'.

    Returns:
        None
    """
    from IPython.display import display  # For better display in Jupyter Notebook
    
    # Extract data from the dictionary
    total_gaps = gaps_data.get('total_gaps', 0)
    largest_gaps = gaps_data.get('largest_gaps', None)
    gaps_df = gaps_data.get('gaps_df', None)
    
    # Print total gaps
    print("\033[1mTotal Number of Gaps:\033[0m", total_gaps)  # Bold title
    
    # Display largest gaps if available
    if largest_gaps is not None:
        print("\n\033[1mTop 10 Largest Gaps:\033[0m")
        display(largest_gaps.style.set_table_styles(
            [{'selector': 'th', 'props': [('font-weight', 'bold')]}]  # Bold column headers
        ).highlight_max(subset=['length_minutes'], color='orange'))  # Highlight largest gap
    
    # Display a preview of all gaps if available
    if gaps_df is not None:
        print("\n\033[1mAll Gaps (Summary - Top 10 Rows):\033[0m")
        display(gaps_df.head(10).style.set_table_styles(
            [{'selector': 'th', 'props': [('font-weight', 'bold')]}]
        ).highlight_max(subset=['length_minutes'], color='orange'))

    # Return the original dictionary for further use if needed
    return gaps_data

gap_analysis = analyse_glucose_gaps(combined_df)
display_gaps = format_gaps_output(gap_analysis)

# Glucose Monitoring Gap Analysis Results

## Overview of Data Gaps
Our analysis identified 5,743 total gaps in glucose monitoring, with durations ranging from 5 minutes to over 40 hours. This comprehensive gap analysis reveals several key patterns:

## Gap Distribution
1. **Major Gaps** (>4 hours):
  - One exceptional gap of 40h 15m (Dec 20-22, 2023)
  - One significant gap of 10h 55m (July 29-30, 2023)
  - One moderate gap of 4h 10m (July 27, 2024)

2. **Medium Gaps** (1-4 hours):
  - Only two gaps in this range
  - Maximum of 1h 55m (May 25, 2024)
  - Minimum of 1h (Aug 23, 2023)

3. **Minor Gaps** (<1 hour):
  - Most common duration: 5 minutes
  - Several gaps between 40-50 minutes
  - Generally well-distributed across the monitoring period

## Critical Observations
1. **Meal Impact**
  - 5 of the top 10 gaps have meals in the preceding 4 hours
  - Most concerning for the 4h+ gaps affecting post-meal analysis

2. **Data Quality**
  - Majority of gaps (>5,000) are brief 5-minute interruptions
  - Only 3 gaps exceed typical meal analysis window (4 hours)
  - Overall excellent data continuity with few significant interruptions

3. **Monitoring Patterns**
  - No seasonal pattern in major gaps
  - Even distribution of minor gaps suggests routine sensor changes
  - Good recovery after interruptions

This analysis supports our decision to proceed with meal impact analysis, as the vast majority of gaps are brief and manageable with our chosen interpolation approach.

# Meal Analysis Configuration and Quality Assessment

## Data Quality Framework
We implement a structured approach for assessing meal data quality and handling glucose data gaps, with clear classification criteria and configurable parameters.

## Quality Categories
1. **Clean Meals**
   - Perfect data quality
   - No gaps in glucose readings
   - Ideal for analysis

2. **Usable Meals**
   - Small gaps (≤15 minutes)
   - Missing data ≤10%
   - Suitable for analysis after interpolation

3. **Borderline Meals**
   - Medium gaps (≤25 minutes)
   - Missing data ≤20%
   - Use with caution

4. **Unusable Meals**
   - Large gaps (>25 minutes)
   - Missing data >20%
   - Exclude from analysis

## Configuration Parameters
- Minimum meal size: 20g carbohydrates
- Post-meal analysis window: 4 hours
- Maximum interpolation: 5 consecutive readings (25 minutes)
- Clear thresholds for gap classification
- Configurable quality criteria

## Processing Workflow
1. Gap Analysis
   - Identifies missing data patterns
   - Calculates gap durations
   - Determines data completeness

2. Quality Assessment
   - Evaluates each meal period
   - Assigns quality categories
   - Marks meals for inclusion/exclusion

3. Data Enhancement
   - Interpolates acceptable gaps
   - Marks interpolated values
   - Preserves data integrity

In [ ]:
class MealQuality(Enum):
    """Enum for meal analysis quality categories"""
    CLEAN = "Clean"          # No gaps
    USABLE = "Usable"        # Small gaps only, ≤10% missing
    BORDERLINE = "Borderline"  # Medium gaps, ≤20% missing
    UNUSABLE = "Unusable"    # Large gaps or >20% missing

@dataclass
class MealAnalysisConfig:
    """Configuration parameters for meal analysis and interpolation"""
    post_meal_hours: int = 4
    usable_gap_mins: int = 15
    max_gap_mins: int = 25
    usable_missing_pct: float = 0.10
    max_missing_pct: float = 0.20
    min_carbs: float = 20.0  # Minimum carbs to consider as meal
    interpolation_limit: int = 5  # Maximum points to interpolate (25 mins at 5-min intervals)

def analyse_meal_data(aligned_df: pd.DataFrame, config: MealAnalysisConfig = None) -> pd.DataFrame:
    """
    Analyses diabetes data for meal analysis suitability and handles missing data interpolation, using 
    MealAnalysisConfig variables if not overwritten and interpolates wheres suitable.
    
    Processes the data in these steps:
    1. Analyses each meal period (4h post-meal by default)
    2. Categorises meal quality based on missing data patterns
    3. Interpolates missing glucose values within acceptable gaps
    4. Marks quality and interpolation status
    
    Args:
        aligned_df: DataFrame with 5-min aligned diabetes data
                   Must have columns: mg_dl, mmol_l, carbs
        config: Optional MealAnalysisConfig object with analysis parameters
    
    Returns:
        DataFrame with:
        - Interpolated glucose values where appropriate
        - Additional columns:
            - has_missing_data: Boolean, True if any missing BG in post-meal period
            - gap_duration_mins: Maximum gap duration in post-meal period
            - missing_pct: Percentage of missing readings in post-meal period
            - meal_quality: Category from MealQuality enum
            - skip_meal: Boolean, True if meal should be excluded from analysis
            - interpolated: Boolean, True if value was interpolated
    """
    if config is None:
        config = MealAnalysisConfig()
    
    # Create copy to avoid modifying original
    df = aligned_df.copy()
    
    # Initialize new columns
    df['has_missing_data'] = False
    df['gap_duration_mins'] = 0.0
    df['missing_pct'] = 0.0
    df['meal_quality'] = None
    df['skip_meal'] = False
    df['interpolated'] = False
    
    # Number of 5-min intervals in post-meal period
    intervals = config.post_meal_hours * 12
    
    # First pass: Analyze gaps and mark meal quality
    meal_rows = df[df['carbs'] > config.min_carbs].index
    
    for meal_time in meal_rows:
        # Get post-meal period
        end_time = meal_time + pd.Timedelta(hours=config.post_meal_hours)
        post_meal = df[meal_time:end_time]
        
        # Skip if we don't have enough future data
        if len(post_meal) < intervals:
            df.loc[meal_time, 'skip_meal'] = True
            df.loc[meal_time, 'meal_quality'] = MealQuality.UNUSABLE.value
            continue
        
        # Analyze missing data
        missing_mask = post_meal['mg_dl'].isna()
        df.loc[meal_time, 'has_missing_data'] = missing_mask.any()
        
        # Calculate missing percentage
        missing_pct = missing_mask.mean()
        df.loc[meal_time, 'missing_pct'] = missing_pct * 100
        
        # Find longest gap
        gap_duration = 0
        current_gap = 0
        
        for is_missing in missing_mask:
            if is_missing:
                current_gap += 5  # 5-minute intervals
                gap_duration = max(gap_duration, current_gap)
            else:
                current_gap = 0
        
        df.loc[meal_time, 'gap_duration_mins'] = gap_duration
        
        # Determine meal quality
        if not missing_mask.any():
            quality = MealQuality.CLEAN
            skip = False
        elif gap_duration <= config.usable_gap_mins and missing_pct <= config.usable_missing_pct:
            quality = MealQuality.USABLE
            skip = False
        elif gap_duration <= config.max_gap_mins and missing_pct <= config.max_missing_pct:
            quality = MealQuality.BORDERLINE
            skip = False
        else:
            quality = MealQuality.UNUSABLE
            skip = True
        
        df.loc[meal_time, 'meal_quality'] = quality.value
        df.loc[meal_time, 'skip_meal'] = skip
    
    # Second pass: Interpolate missing data where appropriate
    # Only interpolate for non-skipped meals
    usable_meals = df[
        (df['carbs'] > config.min_carbs) & 
        (~df['skip_meal'])
    ].index
    
    for meal_time in usable_meals:
        end_time = meal_time + pd.Timedelta(hours=config.post_meal_hours)
        post_meal_idx = df[meal_time:end_time].index
        
        # Interpolate glucose values
        original_mg_dl = df.loc[post_meal_idx, 'mg_dl'].copy()
        original_mmol_l = df.loc[post_meal_idx, 'mmol_l'].copy()
        
        # Use time-based interpolation with limit
        df.loc[post_meal_idx, 'mg_dl'] = df.loc[post_meal_idx, 'mg_dl'].interpolate(
            method='time', 
            limit=config.interpolation_limit
        )
        df.loc[post_meal_idx, 'mmol_l'] = df.loc[post_meal_idx, 'mmol_l'].interpolate(
            method='time', 
            limit=config.interpolation_limit
        )
        
        # Mark interpolated values
        df.loc[post_meal_idx, 'interpolated'] = (
            df.loc[post_meal_idx, 'mg_dl'].notna() & 
            original_mg_dl.isna()
        )
    
    return df

# Meal Quality Statistics Generation

This function computes comprehensive statistics about our meal dataset quality, providing key metrics for assessing data reliability and processing effectiveness.

## Generated Statistics
The function calculates essential metrics across several dimensions:

1. **Volume Metrics**
  - Total number of meals (>20g carbs)
  - Counts per quality category
  - Percentage distribution across categories

2. **Quality Indicators**
  - Average percentage of missing data
  - Average gap duration
  - Proportion of usable meals
  - Number and percentage of interpolated points

## Statistical Output Format
Returns a dictionary containing:
- Raw counts and percentages
- Data completeness metrics
- Interpolation impact measures
- All metrics aligned with MealAnalysisConfig parameters

In [ ]:
def get_meal_statistics(df: pd.DataFrame, config: MealAnalysisConfig = None) -> Dict:
    """
    Calculates statistics about meal quality and data completeness.
    
    Args:
        df: DataFrame with meal analysis columns
        config: MealAnalysisConfig object to ensure consistent carb threshold
               If None, uses default config
    
    Returns:
        Dictionary containing:
        - Total meals analyzed
        - Counts and percentages for each meal quality category
        - Average missing data percentage
        - Distribution of gap durations
        - Interpolation statistics
    """
    if config is None:
        config = MealAnalysisConfig()
        
    meals = df[df['carbs'] > config.min_carbs]
    
    stats = {
        'total_meals': len(meals),
        'quality_counts': meals['meal_quality'].value_counts().to_dict(),
        'quality_percentages': meals['meal_quality'].value_counts(normalize=True).multiply(100).to_dict(),
        'avg_missing_pct': meals['missing_pct'].mean(),
        'avg_gap_duration': meals['gap_duration_mins'].mean(),
        'usable_meals_pct': meals[~meals['skip_meal']].shape[0] / len(meals) * 100,
        'interpolated_points': df['interpolated'].sum(),
        'interpolated_pct': (df['interpolated'].sum() / len(df)) * 100
    }
    
    return stats

In [ ]:
def format_meal_statistics(stats: Dict):
    """
    Pretty-prints meal statistics in a Jupyter Notebook.
    
    Args:
        stats: Dictionary returned by the get_meal_statistics function.
    
    Returns:
        None
    """
    # Create bold headers for clarity
    print("\033[1mMeal Analysis Statistics:\033[0m\n")
    
    # Total meals and averages
    print(f"\033[1mTotal Meals Analyzed:\033[0m {stats['total_meals']}")
    print(f"\033[1mAverage Missing Data Percentage:\033[0m {stats['avg_missing_pct']:.2f}%")
    print(f"\033[1mAverage Gap Duration:\033[0m {stats['avg_gap_duration']:.2f} minutes")
    print(f"\033[1mUsable Meals Percentage:\033[0m {stats['usable_meals_pct']:.2f}%")
    
    # Quality counts and percentages
    print("\n\033[1mMeal Quality Distribution:\033[0m")
    quality_df = pd.DataFrame({
        'Quality': stats['quality_counts'].keys(),
        'Count': stats['quality_counts'].values(),
        'Percentage (%)': stats['quality_percentages'].values()
    })
    quality_df = quality_df.sort_values('Count', ascending=False)
    
    # Apply color to the Count column by using a custom color function
    def color_count(val):
        if val < 5:
            color = 'lightpink'
        elif val < 10:
            color = 'lightblue'
        else:
            color = 'lightgreen'
        return f'background-color: {color}'

    # Apply the color function to the 'Count' column
    quality_df_styled = quality_df.style.map(color_count, subset=['Percentage (%)'])
    
    # Format Percentage column with 2 decimal places
    quality_df_styled = quality_df_styled.format({'Percentage (%)': '{:.2f}%'})
    
    # Display the styled DataFrame
    display(quality_df_styled.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold')]}]
    ))

    # Interpolation statistics
    print("\n\033[1mInterpolation Statistics:\033[0m")
    interpolation_df = pd.DataFrame({
        'Metric': ['Interpolated Points', 'Interpolated Percentage'],
        'Value': [stats['interpolated_points'], f"{stats['interpolated_pct']:.2f}%"]
    })
    display(interpolation_df.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold')]}]
    ).hide(axis="index"))

    print("\n\033[1mAll statistics displayed.\033[0m")

In [ ]:
analysed_df = analyse_meal_data(combined_df) # Run meal quality analysis
stats = get_meal_statistics(analyzed_df)     # Create analysis statistics
display_quality = format_meal_statistics(stats) # Format and print statistics
display_quality

# Meal Analysis Results Summary

## Dataset Overview
Analysis of 1,628 meals (>20g carbs) reveals excellent data quality with high usability for gastroparesis screening:

## Quality Distribution
- **Clean Meals**: 170 (10.44%)
 * Perfect data quality
 * No interpolation needed
- **Usable Meals**: 1,309 (80.41%)
 * Primary analysis dataset
 * Minor gaps successfully interpolated
- **Borderline Meals**: 137 (8.42%)
 * Usable with caution
 * Higher but acceptable gap rates
- **Unusable Meals**: 12 (0.74%)
 * Excluded from analysis
 * Excessive missing data

## Data Quality Metrics
1. **Gap Characteristics**
  - Average gap duration: 4.90 minutes
  - Average missing data: 4.55%
  - Both well within acceptable limits

2. **Interpolation Impact**
  - 1,808 points interpolated
  - Only 1.30% of total readings
  - Minimal data manipulation required

3. **Overall Usability**
  - 99.26% of meals usable for analysis
  - Strong foundation for gastroparesis screening
  - High confidence in data integrity

These results indicate exceptional data quality with minimal need for interpolation, providing a robust dataset for subsequent analysis.

# Dataset Structure and Statistical Summary

## Dataset Properties Analysis
After meal quality analysis and interpolation, let's examine our complete dataset structure and statistical distribution.

## Data Structure
Let's examine:
1. Total records and memory usage
2. Column data types
3. Non-null counts
4. Index structure and timeline coverage

## Statistical Distribution
The describe() output provides key statistics for each column:
1. Central tendency (mean, std)
2. Range and quartiles
3. Treatment frequency and size
4. Glucose value distribution

This verification ensures our processing maintained data integrity while improving usability.

In [ ]:
analysed_df.info()

In [ ]:
analysed_df.describe()

# Diabetes Data Processing and Meal Analysis Summary

## Initial Data Processing
We started with three separate dataframes extracted from xDrip+ CGM management software:
- Blood glucose readings (mg/dl and mmol/l) at approximately 5-minute intervals
- Carbohydrate intake records
- Insulin (bolus and basal) records processed from treatment logs

## Data Alignment and Quality Analysis
1. Aligned all data to regular 5-minute intervals
2. Created a meal analysis framework with configurable parameters:
  - 4-hour post-meal analysis windows
  - Minimum 20g carbs to consider as significant meal
  - Maximum gap tolerance of 25 minutes
  - Maximum 20% missing data allowed
  - Interpolation limit of 25 minutes (5 readings)

## Quality Metrics and Results
Our analysis of 1,628 significant meals (>20g carbs) shows:
- 170 Clean meals (10.44%): Perfect data, no gaps
- 1,309 Usable meals (80.41%): Small gaps only, ≤10% missing
- 137 Borderline meals (8.42%): Medium gaps, ≤20% missing
- Only 12 Unusable meals (0.74%): Large gaps or excessive missing data

The data quality is excellent:
- 99.26% of meals are usable for analysis
- Average missing data is only 4.55%
- Average gap duration is 4.90 minutes
- Only 1.30% of points needed interpolation (1,808 points)

## New Dataset Features
The processed dataframe includes additional columns crucial for gastroparesis screening:
- has_missing_data: Flags any missing BG in 4h post-meal, crucial for response curve integrity
- gap_duration_mins: Longest gap in post-meal period, important for meal absorption analysis
- missing_pct: Percentage of missing readings, validates data quality
- meal_quality: Clean/Usable/Borderline/Unusable, enables filtered analysis
- skip_meal: Boolean for excluding from analysis, maintains data quality standards
- interpolated: Marks interpolated values, ensures transparency in analysis

## Important Notes
1. All original data is preserved - no rows have been removed
2. Small gaps have been interpolated where appropriate
3. Quality markers allow easy filtering for analysis
4. 20g carbohydrate threshold ensures significant meal impacts
5. Data structure supports comprehensive meal response analysis

## Next Steps
While the data is now well-organized and quality-assessed, specific gastroparesis analysis may require:
- Meal size stratification analysis (varying carbohydrate loads)
- Time-of-day impact assessment on gastric emptying
- Insulin timing relationship with meal absorption
- Sensor type and calibration effects on readings
- Additional meal characteristic analysis
- Specific time window selections for different meal types
- Additional derived features for absorption patterns
- Custom filtering based on research requirements

This processed dataset provides a robust foundation for investigating gastric motility patterns through post-meal glucose responses.

In [ ]:
# Save dataframe to CSV file for future use
analysed_df.to_csv('data/processed_data.csv')